Aqui estão as explicações para o funcionamento do notebook Jupyter:

---

### 1. Importações de Bibliotecas e Definições Iniciais

Este trecho de código é responsável por importar as bibliotecas necessárias para o projeto, como `logging`, `os`, `pandas`, `spacy`, `torch`, entre outras. Além disso, são importados módulos específicos do projeto, como `DanteTokenizer` e funções de pré-processamento. 

Também são definidos modelos disponíveis e pré-processadores específicos para cada tipo de dados. 

---
Em resumo, basta definir os valores das variáveis da sessão 2 e executar todas as células em ordem.

In [1]:
import logging
import os
from typing import List, Tuple
import pandas as pd
import spacy
import torch
from dante_tokenizer import DanteTokenizer
from transformers import AutoModelForTokenClassification, AutoTokenizer

from dante_tokenizer.data.preprocessing import split_monetary_tokens, normalize_text, split_enclisis

from preprocessing import *

try:
    nlp = spacy.load("pt_core_news_sm")
except Exception:
    os.system("python -m spacy download pt_core_news_sm")
    nlp = spacy.load("pt_core_news_sm")
dt_tokenizer = DanteTokenizer()

model_choices = {
    "News": "Emanuel/porttagger-news-base",
    "Tweets (stock market)": "Emanuel/porttagger-tweets-base",
    "Oil and Gas (academic texts)": "Emanuel/porttagger-oilgas-base",
    "Multigenre": "Emanuel/porttagger-base",
}
pre_tokenizers = {
    "News": nlp,
    "Tweets (stock market)": dt_tokenizer.tokenize,
    "Oil and Gas (academic texts)": nlp,
    "Multigenre": nlp,
}


c:\Users\felma\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Definições de Variáveis e Configurações

Neste trecho, são definidas variáveis como o modelo padrão a ser utilizado (`default_model`), o nome da coluna de identificação (`id_column`), o nome da coluna de conteúdo (`content_column`), o caminho dos dados de entrada (`data_path`), o caminho de saída (`output_path`), e uma configuração para manter ou substituir contrações (`keep_replace_contraction`). 

Também é configurado o nível de log para `DEBUG`.

In [2]:
default_model = "Tweets (stock market)"
id_column = 'tweet_id'
content_column = 'content'
prefix = 'dante_02'
data_path = 'data/tweets_covid_annotated.csv'
output_path = 'output/tweets_covid_annotated.conllu'
keep_replace_contraction = True

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

### 3. Classe `MyApp`

Esta classe é responsável por carregar o modelo selecionado utilizando a função `load_model`.

In [3]:
class MyApp:
    def __init__(self) -> None:
        self.model = None
        self.tokenizer = None
        self.pre_tokenizer = None
        self.load_model()

    def load_model(self, model_name: str = default_model):
        if model_name not in model_choices.keys():
            logger.error("Selected model is not supported, resetting to the default model.")
            model_name = default_model
        self.model = AutoModelForTokenClassification.from_pretrained(model_choices[model_name])
        self.tokenizer = AutoTokenizer.from_pretrained(model_choices[model_name])
        self.pre_tokenizer = pre_tokenizers[model_name]

myapp = MyApp()

### 4. Função `predict`

Esta função recebe um texto como entrada e faz previsões utilizando o modelo carregado pela instância de `MyApp`. A função tokeniza o texto, faz previsões de tokenização e retorna os tokens, as etiquetas preditas e as pontuações associadas a essas previsões.

In [4]:
def predict(text, logger=None) -> Tuple[List[str], List[str]]:
    doc = myapp.pre_tokenizer(text)
    tokens = [token.text if not isinstance(token, str) else token for token in doc]

    logger.info("Starting predictions for sentence: {}".format(text))
    print("Using model {}".format(myapp.model.config.__dict__["_name_or_path"]))

    input_tokens = myapp.tokenizer(
        tokens,
        return_tensors="pt",
        is_split_into_words=True,
        return_offsets_mapping=True,
        return_special_tokens_mask=True,
        
    )
    output = myapp.model(input_tokens["input_ids"])

    i_token = 0
    labels = []
    scores = []
    for off, is_special_token, pred in zip(
        input_tokens["offset_mapping"][0],
        input_tokens["special_tokens_mask"][0],
        output.logits[0],
    ):
        if is_special_token or off[0] > 0:
            continue
        label = myapp.model.config.__dict__["id2label"][int(pred.argmax(axis=-1))]
        if logger is not None:
            logger.info("{}, {}, {}".format(off, tokens[i_token], label))
        labels.append(label)
        scores.append(
            "{:.2f}".format(100 * float(torch.softmax(pred, dim=-1).detach().max()))
        )
        i_token += 1

    return tokens, labels, scores

### 5. Função `batch_analysis_csv`

Esta função realiza uma análise em lote de um arquivo CSV, onde cada linha do CSV contém um texto a ser analisado. Ela carrega o arquivo CSV, processa os textos, faz previsões para cada texto utilizando a função `predict`, e gera um arquivo de saída no formato CoNLL-U contendo as previsões feitas pelo modelo.

In [5]:
def batch_analysis_csv(id_column: str, content_column: str, data_path: str, prefix:str, output_path: str, keep_replace_contraction: bool):
    df = pd.read_csv(data_path)
    ids = df[id_column]
    texts = df[content_column]

    texts = texts.replace(r'\\n', ' ', regex=True) # remover '\n' mas não por espaço
    texts = texts.apply(lambda x : x.strip()) # remover espaços excedentes

    conllu_output = []

    for id, sent in zip(ids, texts):
        #print(sent)
        
        conllu_output.append("# sent_id = {}_{}\n".format(prefix, id))
        conllu_output.append("# text = {}\n".format(sent))
        
        tokens, labels, _ = predict(sent, logger)
        tokens_labels = list(zip(tokens, labels))

        for j, (token, label) in enumerate(tokens_labels):
            #print(tokens_labels[j][0])
            try:
                contr = tokens_labels[j][0] + ' ' + tokens_labels[j+1][0]
                for expansion in expansions.keys():
                    replace_str = expansions[expansion]
                    match = re.match(expansion, contr, re.IGNORECASE)
                    expansion = replace_keep_case(expansion, replace_str, contr)
                    if match is not None:
                        conllu_output.append("{}\t{}".format(str(j+1)+'-'+str(j+2), expansion) + "\t_" * 8 + "\n")
                        break
                conllu_output.append("{}\t{}\t_\t{}".format(j + 1, token, label) + "\t_" * 6 + "\n")

            except IndexError:
                conllu_output.append("{}\t{}\t_\t{}".format(j + 1, token, label) + "\t_" * 6 + "\n")
            
        conllu_output.append("\n")

    with open(output_path, 'w', encoding='utf-8') as out_f:
        out_f.writelines(conllu_output)

### 6. Execução do Processamento em Lote

Neste trecho, a função `batch_analysis_csv` é chamada com os parâmetros previamente definidos. Isso inicia o processo de análise em lote dos dados contidos no arquivo CSV especificado, utilizando o modelo padrão e as configurações definidas anteriormente.

In [6]:
batch_analysis_csv(id_column, content_column, data_path, prefix, output_path, keep_replace_contraction)

Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model Emanuel/porttagger-tweets-base
Using model